In [10]:
import numpy as np
import tensorflow as tf


# Percorsi ai file
X = np.load("dataset_75/images.npy")
y = np.load("dataset_75/labels.npy")

X = X.astype("float32") / 255.0

X = X[..., np.newaxis]

print(X.shape)  # es. (18000, 150, 150) o (18000, 75, 75)
print(y.shape)  # es. (18000, 2)



(18000, 75, 75, 1)
(18000, 2)


In [11]:
label_range=30 #minutes the are togheter in the same class
number_of_classifiers=12*60//label_range

label=y[:,0]+y[:,1]/60

print(label[17999])
print(y[17999])
print(number_of_classifiers)

11.983333333333333
[11 59]
24


In [12]:
from sklearn.model_selection import train_test_split

X_traina, X_test, y_traina, y_test = train_test_split(X, label, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_traina, y_traina, test_size=y_test.shape[0], random_state=42, shuffle=True)

print(X_test.shape)
print(X_val.shape)
print(X_train.shape)
print(X_train[0].shape)

(1800, 75, 75, 1)
(1800, 75, 75, 1)
(14400, 75, 75, 1)
(75, 75, 1)


In [13]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import layers, models, regularizers
from keras.optimizers import AdamW
from keras.callbacks import ReduceLROnPlateau


In [14]:
class PeriodicOutput(layers.Layer):
    def call(self, inputs):
        return (tf.math.sin(inputs) + 1) * 6  

model_base = models.Sequential([
    keras.layers.Conv2D(32, (5,5), padding="same", input_shape=X_train[0].shape),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D(),
    #keras.layers.Dropout(0.3),

    keras.layers.Conv2D(64, (3,3), padding="same", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.AveragePooling2D(2,2),
    #keras.layers.Dropout(0.3),

    keras.layers.Conv2D(128, (3,3), padding="same", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.AveragePooling2D(2,2),
    #keras.layers.Dropout(0.3),

    keras.layers.Conv2D(256, (3,3), padding="same", activation="relu"),
    keras.layers.BatchNormalization(),
    #keras.layers.AveragePooling2D(2,2),
    #keras.layers.Dropout(0.3),    

    keras.layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.3),
    keras.layers.Dense(1),
    PeriodicOutput()
])

/vol/home/s4949404/miniconda3/envs/IDL/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
def custom_accuracy(y_true, y_pred):
    diff = tf.minimum(tf.abs(y_true - y_pred), tf.abs(12 - tf.abs(y_true - y_pred)))
    return tf.reduce_mean(tf.cast(diff < 0.5, tf.float32))

model_base.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=5e-5),
    loss="mse",
    metrics=[custom_accuracy]
)

In [16]:
# Reduce LR when validation loss stops improving
lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=5, min_lr=1e-7
)

checkpoint = keras.callbacks.ModelCheckpoint(
    "best_model.keras",
    save_best_only=True
)

early_stopping=keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

In [17]:
history_base = model_base.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[lr_scheduler,checkpoint,early_stopping]
)


Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - custom_accuracy: 0.0827 - loss: 10.3171 - val_custom_accuracy: 0.0857 - val_loss: 17.6373 - learning_rate: 1.0000e-04
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - custom_accuracy: 0.0832 - loss: 8.5117 - val_custom_accuracy: 0.0897 - val_loss: 44.2781 - learning_rate: 1.0000e-04
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - custom_accuracy: 0.0832 - loss: 7.7169 - val_custom_accuracy: 0.0835 - val_loss: 20.6634 - learning_rate: 1.0000e-04
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - custom_accuracy: 0.0836 - loss: 7.1997 - val_custom_accuracy: 0.0808 - val_loss: 39.8434 - learning_rate: 1.0000e-04
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - custom_accuracy: 0.0839 - loss: 6.6549 - val_custom_accuracy: 0.0821 - val_loss: 31.4069 - learning_rate: 1.0000e-04
Epoch 6/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - custom_accuracy: 0.0836 - loss: 6.1740 - val_custom_accuracy: 0.0838 - val_los

In [18]:
# Carichiamo il modello salvato con i pesi migliori
best = keras.models.load_model("best_model.keras")

# Valutiamo sul test set
test_loss, test_acc = best.evaluate(X_train, y_train)
print(f"Loss sul train set: {test_loss:.4f}")
print(f"Accuratezza sul train set: {test_acc:.4f}")

# Valutiamo sul test set
test_loss, test_acc = best.evaluate(X_test, y_test)
print(f"Loss sul test set: {test_loss:.4f}")
print(f"Accuratezza sul test set: {test_acc:.4f}")

TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 131698008186160}, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 75, 75, 1], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer_1'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_4', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 32, 'kernel_size':

In [ ]:
y_pred = model_base.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred_classes)
ConfusionMatrixDisplay(cm).plot()




In [ ]:
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history_base.history["accuracy"], label="Train")
plt.plot(history_base.history["val_accuracy"], label="Validation")
plt.title("Accuratezza")
plt.xlabel("Epoca")
plt.ylabel("Accuracy")
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history_base.history["loss"], label="Train")
plt.plot(history_base.history["val_loss"], label="Validation")
plt.title("Loss")
plt.xlabel("Epoca")
plt.ylabel("Loss")
plt.legend()

plt.show()